## R - Choropleth Maps


In this notebook, we will see examples of choropleth maps using ggplot.

Maps are represented by very detailed polygons in vector form. Let's first start with simple polygons. 

In [ ]:
library(ggplot2)
library(maps)
library(ggmap)
library(mapproj)


# To color polygons, we usually need two data frames; 
# one contains the coordinates of each polygon (positions),  
# and the other one contains the values associated with each polygon (values).  
# A common variable links the two together (ids).

# Made up ids; these connect values to polygons
ids <- factor(c("1.1", "2.1", "1.2", "2.2", "1.3", "2.3"))

# Some made up values: the colors will represent these values 
values <- data.frame(id = ids, value = c(3, 3.1, 3.1, 3.2, 3.15, 3.5))

# coordinates of some polygons
positions <- data.frame(id = rep(ids, each = 4),
  x = c(2, 1, 1.1, 2.2, 1, 0, 0.3, 1.1, 2.2, 1.1, 1.2, 2.5, 1.1, 0.3,
  0.5, 1.2, 2.5, 1.2, 1.3, 2.7, 1.2, 0.5, 0.6, 1.3),
  y = c(-0.5, 0, 1, 0.5, 0, 0.5, 1.5, 1, 0.5, 1, 2.1, 1.7, 1, 1.5,
  2.2, 2.1, 1.7, 2.1, 3.2, 2.8, 2.1, 2.2, 3.3, 3.2))

head(values)
head(positions)

In [ ]:
# Let's draw and color them: map_id=id is what associates the two data frames
ggplot(data = values, aes(fill = value)) +
# common variable is id
geom_map(aes(map_id = id), map = positions) +
expand_limits(positions)

#### Now, let's do the same for US map to visualize the crime data set. 
We need to create lowercase state names, and they will serve as the common variable between the map data and the crime data set. 
We will use the R built-in data set for **USArrests**

In [ ]:
help(USArrests) # review information about this built-in R data set.

In [ ]:
head(USArrests)

# First create lowercase state names, because these will be the "IDs" that will connect 
# the crimes data to the map data that also contains lowercase state names and their polygon coordinates. 
crimes <- data.frame(state = tolower(rownames(USArrests)), USArrests)
# convert arrest variables in the columns to a single column to enable faceting later. 
crimesm <- reshape2::melt(crimes[,c(1,2,3,5)], id = 1)
head(crimes)
head(crimesm)
tail(crimesm)

**Note:** The about code used the `melt` function to do a transform of the data.  
If you look close, the column heading `Murder` has dropped down to be a value in the column variable.
Similarly, the other columns have done this as well, shown with the `tail` of the `melt`ed data.
This effectively converts the column names to be factors.
 * [Blog post on Melt](https://www.r-bloggers.com/melt/)
 
We can review the characteristics of the new dataframe: 

In [ ]:
str(crimesm)

In [ ]:
# Get the map data for US states
states_map <- map_data("state")

In [ ]:
# "state" is the common variable (ids)
ggplot(data=crimes, aes(fill=Murder)) + 
# color will encode Murder, state boundaries will be white and size of 0.1 
geom_map(map=states_map, aes(map_id=state), color="white", size=0.1) +
# coordinate limits of the map 
expand_limits(x = states_map$long, y = states_map$lat) + 
# useful map projection 
coord_map("polyconic") +
# remove grid and coord info 
theme_void() 

### Using categorical variables to split plots into facets

In [ ]:
# Similarly, we can do small multiples 
ggplot(data=crimesm, aes(fill = value)) +
geom_map(map = states_map, aes(map_id = state), color="white", size=0.1) +
expand_limits(x = states_map$long, y = states_map$lat) +
facet_wrap(~variable, ncol=2) + 
coord_map("polyconic") +
theme_void()

In [ ]:
# Let's get another data set 
agex <- read.csv("/dsa/data/all_datasets/2011_us_ag_exports.csv")
head(agex)

In [ ]:
library(dplyr) 
# Let's create lowercase state names 
agex <- mutate(agex, state = tolower(state))
head(agex)

In [ ]:
# Let's draw and color them
ggplot() +
geom_map(data=agex, map = states_map, aes(fill = total.exports, map_id = state), color="white", size=0.3) +
expand_limits(x = states_map$long, y = states_map$lat) +
coord_map("polyconic") +
theme_void() +
ggtitle("US Agriculture Exports") 

In [ ]:
# Pick a few variables
sub <- agex[, c(2,14,15,16,17)]
head(sub)
# melt them into a single variable
subm <- reshape2::melt(sub, id = 1)
head(subm)

In [ ]:
ggplot(subm, aes(map_id = state)) +
geom_map(aes(fill = value), map = states_map, color="white", size=0.1) +
expand_limits(x = states_map$long, y = states_map$lat) + 
scale_fill_continuous(low='thistle2', high='darkblue') +
coord_map("polyconic") + 
facet_wrap(~variable, ncol=2) +
theme_void() +
theme(legend.position="None") + 
ggtitle("US Agriculture Exports") 


### Doing a little more numerical carpentry.

We can transform the data from counts to percentages and more intuitive comparison across the factors.

In [ ]:
# Let's compare percentages 
sub <-mutate(sub, corn=corn/sum(corn))
sub <-mutate(sub, wheat=wheat/sum(wheat))
sub <-mutate(sub, cotton=cotton/sum(cotton))
sub <-mutate(sub, total.veggies=total.veggies/sum(total.veggies))
subm <- reshape2::melt(sub, id = 1)

ggplot(subm, aes(map_id = state)) +
geom_map(aes(fill = value), map = states_map, color="white", size=0.1) +
expand_limits(x = states_map$long, y = states_map$lat) + 
scale_fill_continuous(low='thistle2', high='darkblue') +
coord_map("polyconic") + 
facet_wrap(~variable, ncol=2) +
theme_void() +
theme(legend.position="None") + 
ggtitle("US Agriculture Exports") 
